(СТРОКИ, СТОЛБЦЫ)

## 1. Объявление функций для отбора строк, столбцов CUR-реализация и метрика
---

Изначально возикала ошибка для ф-ии `solve_triangular` из-за того, что матрица в аргументе была сингулярной, решением стало использовать матричное перемножение псевдообратной матрицы метрика процента совпадений и норма разницы несколько упали, но незначительно

In [1]:
from scipy.linalg import pinv
import cv2
import matplotlib.pyplot as plt
import pandas as pd

import numpy as np
from scipy.linalg import lu
from scipy.linalg import solve_triangular

In [2]:
def maxvol(A, e=1.05, k=100, test=False):
    """Compute the maximal-volume submatrix for given tall matrix.

    Args:
        A (np.ndarray): tall matrix of the shape [n, r] (n > r).
        e (float): accuracy parameter (should be >= 1). If the parameter is
            equal to 1, then the maximum number of iterations will be performed
            until true convergence is achieved. If the value is greater than
            one, the algorithm will complete its work faster, but the accuracy
            will be slightly lower (in most cases, the optimal value is within
            the range of 1.01 - 1.1).
        k (int): maximum number of iterations (should be >= 1).

    Returns:
        (np.ndarray, np.ndarray): the row numbers I containing the maximal
        volume submatrix in the form of 1D array of length r and coefficient
        matrix B in the form of 2D array of shape [n, r], such that
        A = B A[I, :] and A (A[I, :])^{-1} = B.

    Note:
        The description of the basic implementation of this algorithm is
        presented in the work: Goreinov S., Oseledets, I., Savostyanov, D.,
        Tyrtyshnikov, E., Zamarashkin, N. "How to find a good submatrix".
        Matrix Methods: Theory, Algorithms And Applications: Dedicated to the Memory of Gene Golub (2010): 247-256.

    """
    n, r = A.shape

    if n <= r:
        raise ValueError('Input matrix should be "tall"')

    P, L, U = lu(A, check_finite=False)
    I = P[:, :r].argmax(axis=0)
    if test == False:
        Q = solve_triangular(U, A.T, trans=1, check_finite=False)
        B = solve_triangular(L[:r, :], Q, trans=1, check_finite=False,
            unit_diagonal=True, lower=True).T
    else:
        Q = np.dot(np.linalg.pinv(U), A.T)
        B = np.dot(np.linalg.pinv(L[:r, :]), Q).T

    for _ in range(k):
        i, j = np.divmod(np.abs(B).argmax(), r)
        if np.abs(B[i, j]) <= e:
            break

        I[j] = i

        bj = B[:, j]
        bi = B[i, :].copy()
        bi[j] -= 1.

        B -= np.outer(bj, bi / B[i, j])

    return I, B

def maxsumcolumns(A,r):
    columns_sums = A.sum(axis=0)

    columns_index = np.argsort(columns_sums)[-r:][::-1]

    return columns_index

def cur_decomposition(A, test=False, rank=0):
    """
    Выполняет CUR-разложение для матрицы A.
    
    Параметры:
    A : numpy.ndarray : Исходная матрица размера (m, n)
    c : int : Количество столбцов, которые будут выбраны в C
    r : int : Количество строк, которые будут выбраны в R
    
    Возвращает:
    C : numpy.ndarray : Подматрица столбцов из A размера (m, c)
    U : numpy.ndarray : Матрица соединения размера (c, r)
    R : numpy.ndarray : Подматрица строк из A размера (r, n)
    """
    
    # Шаг 2: Выбор случайных строк для матрицы R
    if rank == 0:
        rank = np.linalg.matrix_rank(A)
    I, B = maxvol(A, test=test)
    I = I[:rank]
    R = A[I, :]

    column_indices = maxsumcolumns(A,rank)
    C = A[:, column_indices]
    
    W = A[np.ix_(I, column_indices)]
    
    U = pinv(W)
    
    return C, U, R


In [5]:
def percent_metric(A_true, A_approx):
    A_approx = np.round(A_approx,0)
    return (A_true==A_approx).sum()/(A_true.shape[0]*A_true.shape[1])

# 2. Тест на случайно сгенерированной матрице
---

In [4]:
A = np.random.randint(0,2,[1200,700])
print("A rank:", np.linalg.matrix_rank(A))
print(f"A shape: {A.shape}")

A rank: 700
A shape: (1200, 700)


In [11]:
A = np.array([
    [0, 0, 1, 0, 1, 1, 1],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 1, 0, 1, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 0, 1],
    [0, 1, 1, 1, 0, 0, 1],
    [1, 0, 1, 0, 0, 1, 1],
    [1, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 0, 0, 0],
    [1, 1, 0, 0, 1, 0, 0],
    [1, 0, 0, 1, 0, 0, 0],
    [1, 1, 0, 0, 1, 1, 1],
    [0, 1, 1, 0, 1, 0, 0],
    [0, 1, 1, 1, 1, 1, 0],
    [0, 0, 1, 1, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1],
    [1, 0, 0, 1, 0, 0, 1]
])

In [14]:
C,G,R = cur_decomposition(A, test=True)

approximation = C@G@R

print(f"C shape: {C.shape}")
print(f"G shape: {G.shape}")
print(f"R shape: {R.shape}")

print(f"approximation shape: {approximation.shape}")
print(np.linalg.norm(A-approximation))
print(percent_metric(A, approximation))

C shape: (20, 7)
G shape: (7, 7)
R shape: (7, 7)
approximation shape: (20, 7)
2.8722813232690143
0.9357142857142857


In [10]:
new_user = np.random.randint(0,2,700)
films = np.sum(new_user)

R = np.vstack([R, new_user])

# 3. Проверка работоспособности на реальных данных
---

In [3]:
%%time
data = pd.read_csv('data.csv', sep=';')
data = data.to_numpy()
data = data[~np.all(data == 0, axis=1)]
print(f'размер матрицы: {data.shape}')
print(f'ранк матрицы: {np.linalg.matrix_rank(data)}')

размер матрицы: (9723, 610)
ранк матрицы: 610
CPU times: total: 4.62 s
Wall time: 8.3 s


In [15]:
np.random.seed(42)  # Для воспроизводимости
data = np.random.randint(0, 2, size=(70, 25))

# Названия фильмов
movies = [f"Movie {i+1}" for i in range(25)]

# Индексы пользователей
users = [f"User {i+1}" for i in range(70)]

# Создаем DataFrame
UI_matrix = pd.DataFrame(data, index=users, columns=movies)

data = UI_matrix.to_numpy()

In [16]:
C,G,R = cur_decomposition(data, test=True)

approximation = C@G@R

print(f"C shape: {C.shape}")
print(f"G shape: {G.shape}")
print(f"R shape: {R.shape}")

print(f"approximation shape: {approximation.shape}")
print(f'Норма разности: {np.linalg.norm(data-approximation)}')
print(f'Процент совпадений: {np.round(percent_metric(data, approximation)*100,2)}%')

C shape: (70, 25)
G shape: (25, 25)
R shape: (25, 25)
approximation shape: (70, 25)
Норма разности: 5.362937891647742
Процент совпадений: 99.49%


# 4. Система рекомендации на основе разложения ver.1
---

In [19]:
unseen = np.where(data[12]==0)[0]

In [7]:
def recomendation1(user, A, Ap, n=5):
    unseen = np.where(A[user]==0)[0]

    unseen_movie_scores = Ap[user, unseen]

    recommendation = unseen[np.argsort(unseen_movie_scores)[::-1][:n]]

    return recommendation, unseen_movie_scores

In [17]:
rec, scores = recomendation1(12, data, approximation,n=20)

In [18]:
len(rec)

11

In [39]:
np.sort(scores)[::-1]

array([ 1.16575188e+00,  1.10426019e+00,  1.01351298e+00,  9.89781767e-01,
        9.58904144e-01,  9.52781473e-01,  9.37363160e-01,  9.36854078e-01,
        8.72624839e-01,  8.62119969e-01,  8.46938578e-01,  8.26445415e-01,
        8.09203378e-01,  7.97977917e-01,  7.96013871e-01,  7.89892880e-01,
        7.87880222e-01,  7.85856172e-01,  6.79082107e-01,  6.76099426e-01,
        6.70461865e-01,  6.56217349e-01,  6.52906394e-01,  6.27764828e-01,
        6.04235067e-01,  6.01263517e-01,  5.88276640e-01,  5.87087046e-01,
        5.86144072e-01,  5.73322285e-01,  5.72891203e-01,  5.72455754e-01,
        5.67729265e-01,  5.65208388e-01,  5.51492275e-01,  5.43733175e-01,
        5.36584496e-01,  5.34984054e-01,  5.08174155e-01,  4.99186615e-01,
        4.93674560e-01,  4.83672675e-01,  4.77931635e-01,  4.74323746e-01,
        4.68525249e-01,  4.42681731e-01,  4.37894338e-01,  4.30393546e-01,
        4.27017966e-01,  4.17452533e-01,  4.13854968e-01,  4.08779700e-01,
        4.07270772e-01,  